In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-failure-dataset/heart.csv


In [2]:
data= pd.read_csv("/kaggle/input/heart-failure-dataset/heart.csv")

In [3]:
print(data.head(15))

    Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  \
0    40   M           ATA        140          289          0     Normal   
1    49   F           NAP        160          180          0     Normal   
2    37   M           ATA        130          283          0         ST   
3    48   F           ASY        138          214          0     Normal   
4    54   M           NAP        150          195          0     Normal   
5    39   M           NAP        120          339          0     Normal   
6    45   F           ATA        130          237          0     Normal   
7    54   M           ATA        110          208          0     Normal   
8    37   M           ASY        140          207          0     Normal   
9    48   F           ATA        120          284          0     Normal   
10   37   F           NAP        130          211          0     Normal   
11   58   M           ATA        136          164          0         ST   
12   39   M           ATA

In [4]:
data.drop(["FastingBS","ExerciseAngina","ST_Slope"], axis=1, inplace=True)

In [5]:
X= data.drop(["Sex","ChestPainType","RestingECG","HeartDisease"], axis=1)
y= data["HeartDisease"]

In [6]:
print(X.head(10))

   Age  RestingBP  Cholesterol  MaxHR  Oldpeak
0   40        140          289    172      0.0
1   49        160          180    156      1.0
2   37        130          283     98      0.0
3   48        138          214    108      1.5
4   54        150          195    122      0.0
5   39        120          339    170      0.0
6   45        130          237    170      0.0
7   54        110          208    142      0.0
8   37        140          207    130      1.5
9   48        120          284    120      0.0


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [8]:
X_train, X_test, y_train, y_test= train_test_split(X, y, train_size=0.8, random_state=42)

**choosing best k for knn**

In [9]:
test_accuracies={}
k= np.arange(3,16)

In [10]:
print(k)

[ 3  4  5  6  7  8  9 10 11 12 13 14 15]


In [11]:
for i in k:
    knn= KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    accuracy = knn.score(X_test, y_test)
    test_accuracies[i]= accuracy

In [12]:
print(test_accuracies)

{3: 0.657608695652174, 4: 0.6793478260869565, 5: 0.7065217391304348, 6: 0.6684782608695652, 7: 0.6684782608695652, 8: 0.6413043478260869, 9: 0.6684782608695652, 10: 0.6413043478260869, 11: 0.6521739130434783, 12: 0.6467391304347826, 13: 0.6739130434782609, 14: 0.6630434782608695, 15: 0.6630434782608695}


In [13]:
knn_best= KNeighborsClassifier(n_neighbors=5)
knn_best.fit(X_train,y_train)
y_pred= knn_best.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.63      0.71      0.67        77
           1       0.77      0.70      0.74       107

    accuracy                           0.71       184
   macro avg       0.70      0.71      0.70       184
weighted avg       0.71      0.71      0.71       184



**improving my model using Grid Search & cross-validation**

In [14]:
from sklearn.model_selection import GridSearchCV, KFold

In [15]:
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': np.arange(1, 25),
              'metric': ['euclidean', 'manhattan', 'minkowski']
             }

In [16]:
knn_cv=GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')
knn_cv.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan', 'minkowski'],
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])},
             scoring='accuracy')

In [17]:
print(knn_cv.best_params_)
print("Best cross-validation score:", knn_cv.best_score_)

{'metric': 'euclidean', 'n_neighbors': 10}
Best cross-validation score: 0.7355794150314698


In [18]:
best_knn= knn_cv.best_estimator_

In [19]:
y_pred= best_knn.predict(X_test)

In [20]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.56      0.70      0.62        77
           1       0.74      0.60      0.66       107

    accuracy                           0.64       184
   macro avg       0.65      0.65      0.64       184
weighted avg       0.66      0.64      0.64       184



In [21]:
data.drop(["Sex","RestingECG"], axis=1, inplace=True)

In [22]:
print(data.head(10))

   Age ChestPainType  RestingBP  Cholesterol  MaxHR  Oldpeak  HeartDisease
0   40           ATA        140          289    172      0.0             0
1   49           NAP        160          180    156      1.0             1
2   37           ATA        130          283     98      0.0             0
3   48           ASY        138          214    108      1.5             1
4   54           NAP        150          195    122      0.0             0
5   39           NAP        120          339    170      0.0             0
6   45           ATA        130          237    170      0.0             0
7   54           ATA        110          208    142      0.0             0
8   37           ASY        140          207    130      1.5             1
9   48           ATA        120          284    120      0.0             0


In [23]:
data_encoded= pd.get_dummies(data, dtype=int)

In [24]:
print(data_encoded.head(10))

   Age  RestingBP  Cholesterol  MaxHR  Oldpeak  HeartDisease  \
0   40        140          289    172      0.0             0   
1   49        160          180    156      1.0             1   
2   37        130          283     98      0.0             0   
3   48        138          214    108      1.5             1   
4   54        150          195    122      0.0             0   
5   39        120          339    170      0.0             0   
6   45        130          237    170      0.0             0   
7   54        110          208    142      0.0             0   
8   37        140          207    130      1.5             1   
9   48        120          284    120      0.0             0   

   ChestPainType_ASY  ChestPainType_ATA  ChestPainType_NAP  ChestPainType_TA  
0                  0                  1                  0                 0  
1                  0                  0                  1                 0  
2                  0                  1                  0

In [25]:
X=data_encoded.drop(["HeartDisease"], axis=1)
y=data_encoded["HeartDisease"]

**Time for scaling our data**

In [26]:
from sklearn.preprocessing import StandardScaler

In [27]:
scaler= StandardScaler()

In [28]:
X_train_scaled= scaler.fit_transform(X_train)

In [29]:
X_test_scaled = scaler.transform(X_test) 

In [30]:
knn_cv=GridSearchCV(knn, param_grid, cv=15, scoring='accuracy', n_jobs=-1)
knn_cv.fit(X_train_scaled, y_train)

GridSearchCV(cv=15, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['euclidean', 'manhattan', 'minkowski'],
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])},
             scoring='accuracy')

In [31]:
print(knn_cv.best_params_)
print("Best cross-validation score:", knn_cv.best_score_)

{'metric': 'euclidean', 'n_neighbors': 14}
Best cross-validation score: 0.7956349206349207


In [32]:
knn_best=knn_cv.best_estimator_

In [33]:
y_pred=knn_best.predict(X_test_scaled)

In [34]:
print(X_test_scaled[1])

[-0.06959099  0.64776533  2.94347064 -0.24409275 -0.83628643]


In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.65      0.86      0.74        77
           1       0.87      0.67      0.76       107

    accuracy                           0.75       184
   macro avg       0.76      0.77      0.75       184
weighted avg       0.78      0.75      0.75       184



In [36]:
from sklearn.metrics import confusion_matrix

In [37]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Confusion Matrix:
[[66 11]
 [35 72]]
